# Comparing PR Handling in FOSS Commununities

In this notebook we study how different Free & Open-Source Software (FOSS) communities handle Pull Requests (PRs). In particular, we compare how _protective_, _equitable_ and _lenient_ FOSS communities are in handling PRs. [NOTE: Maybe here some examples of what each of these categories mean at a high level]. 

To this end, we have collected information related to PR handling from members of the following FOSS communities [NOTE: reference to the study]:

* FOSSASIA
* Odoo
* DuckDuckGo
* Linux Kernel
* Coala
* ROS
* Plone
* ReactJS
* AngularJS
* NodeJS
* OpenGenus
* Mozilla
* OpenSUSE
* jQuery
* Apache
* Other


## Classification

We are interested in comparing how _protective_, _equitable_ and _lenient_ the different communities are. Below we precisely describe the criteria to classify a community within a category.

### Protective

A community is classified as “Protective” when the response from this community is positive in at least one of these variables V28 (“I don't consider a pull request/patch, unless I trust the contributor”), V29 (“I don't consider a pull request/patch, unless the contributor is reliable.”), and V30 (“I don't consider a pull request/patch, unless I have a strong relationship with the contributor.”). It is possible that positive evaluations of V27 (“In general I  say no to most pull requests (PR)/patches”) can be included in this evaluation too.

### Equitable

A community is classified as “Equitable” when the response from this community is positive in at least one of these variables V31 (“I assess every pull request/patch in the same manner irrespective of the contributor.”), V32 (“I assess pull requests/patches purely on technical grounds.”).  The equitability in community is expected to find positive answers on V31, V32 which have different tone compared to V33 (“I never say no to a pull request/patch. If the quality of the PR/patch is not mergeable, then I mentor the contributor to elevate his/her PR/patch to a mergeable state.”) in question.

### Lenient

A community is classified as “Lenient” when the response from this community is positive V33 (“I never say no to a pull request/patch. If the quality of the PR/patch is not mergeable, then I mentor the contributor to elevate his/her PR/patch to a mergeable state.”).

### Answers 

The participants in the survey selected a mark from the likert scale below to answer each question:

1. Strongly Agree
2. Agree
3. Neutral
4. Disagree
5. Strongly Diasgree


**Remark**: In this study we consider that a community is _positive_ on a variable if the questions of the community are concentrated on strongly agree, agree or neutral. [NOTE: Perhaps here we should say why we choose this threshold]

### Summary

This table summaries the variables we study for each category. For a category $c \in \{\mathit{protective, equitable, lenient}\}$ and the set of variables for that category (denoted as $V_c$), we say that a community is $c$ iff it is positive (see remark above) to either of the questions in $V_c$. For instance, we say that a community is _lenient_ iff it is positive to either _V31_ or _V32_.


| Category   | ID   | Variable                                                                                                           |
|------------|------|--------------------------------------------------------------------------------------------------------------------|
| Protective |      |                                                                                                                    |
|            | V28  | I don't consider a pull request/patch, unless I trust the contributor.                                             |
|            | V29  | I don't consider a pull request/patch, unless the contributor is reliable.                                         |
|            | V30  | I don't consider a pull request/patch, unless I have a strong relationship with the contributor.                   |
|            | V27^ | In general I  say no to most pull requests (PR)/patches”) can be included in this evaluation too.                  |
| Equitable  |      |                                                                                                                    |
|            | V31  | I assess every pull request/patch in the same manner irrespective of the contributor.                              |
|            | V32  | I assess pull requests/patches purely on technical grounds.                                                        |
| Lenient    |      |                                                                                                                    |
|            | V33  | I never say no to a pull request/patch. I mentor the contributor to elevate his/her PR/patch to a mergeable state. |

^ Variable V27 is optional in comparing protectiveness. However, it can be used to get additional information (see [Protective](#Protective) above).


---

[NOTE: Editing the table above in codelab is a pain. The editor provides no support whatsoever for editing markdown tables. My current approach is to edit the table in emacs and copy/paste it here. If you would like to edit the table, I recommend that you copy the table to a markdown editor with support for tables, make some edits, and then copy it back.]

[NOTE: The summary is written in a language that it is useful for me. But my immediate guess is that the TSE community will not appreciate this formulation. So I leave it to you guys to present it in a more TSE friendly manner. Nevertheless, I find the table clarifying.]

## Hypotheses

In what follows we analyze the veracity of the following hypotheses:

* **H1:** The Linux Kernel community adopts a protective style of governance for its code change process.

* **H2:** The FOSSASIA community adopts an equitable style of governance for its pull request process.

* **H3:** The Odoo community adopts an equitable style of governance for its pull request process.

* **H4:** The DuckDuckGo community adopts a linient style of governance for its pull request process.

* **H5:** The Coala community adopts a linient style of governance for its pull request process.

* **H6:** Each FOSS community adopts a governance style, either protective, equitable or linient, for its pull request process.

* **H7 (BONUS):** The Coala Community is more lenient than the Linux Kernel Community

## Bayesian Data Analysis and Hypothesis Testing

In order to test the hypoteses above (see [Hypotehses](#Hypoteses)), we use Bayesian inference to estimate the underlying distributions of the answers that each FOSS community gives to each of the variables defined above.

We assume that the answers to the questions are [normaly distributed](https://en.wikipedia.org/wiki/Normal_distribution) over the possible answers (see [answers](#Answers)). Hence, we use a Bayesian model with normal distributions as a likelihood functions—i.e., the distribution of the data in Bayesian jargon. We set a very uninformative and flexible prior so that we let the inference engine obtain the underlying distribuion of the answers for each community.

Once we have obtained the distributions for each variable of each community we use different techniques to test our hypotheses. These techniques will be discussed in detailed below.

We use PyMC3 to perform our analyses.

In what follows, we first load the data and inialize the libraries used for the analysis, and, later, we present a section per hypothesis describing the model and the code to test the veracity of the hypothesis.

###  Initialization

**Little technical note:** In case you have problems with arviz+pymc3. Just run the cell below.

You will be abel to detect issues if running the imports cell (two cells below) gives you an error.

In [ ]:
!pip install arviz
!pip install git+https://github.com/pymc-devs/pymc3

#### Imports

Here we simply import all the libraries we need for our analysis.

[NOTE: If you think it is necessary I can explain what each library is for]

In [ ]:
import numpy as np
import pandas as pd
import arviz as az
import pymc3 as pm
from matplotlib import pyplot as plt

### Loading the data

We read the data from the pseudonymized data set and filter it by
community. The dataset contains the answers of each participants to questions (variables) from V27 to V33. Each cell in the dataset contains an integer value in $\{-1\} \cup [1,5]$ where $-1$ means that the participant didn't answer the question, $1$ means strongly agree, $2$ agree, and so on.

In order to protect the identity of the participants we have striped out identifiable information from the original dataset. Concretely, unique identfiers and textual answers. This modification has no effect in the results as those data are irrelevant.

Run the cell below to get a preview of the dataset.

In [ ]:
df = pd.read_csv('pseudonymized-data.csv')

### In case the file 'pseudonymized-data.csv' disappears, comment the lines above, and use the code below to upload it again
# from google.colab import files
# import io
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded['pseudonymized-data.csv']))

# Show a preview of the data
df

### H1: The Linux Kernel community adopts a protective style of governance for its code change process

Here we analyze the veracity of hypothesis H1.

#### Data

Here we simply load the data we analyze, i.e., answers to question V33 by the Coala community.

In [ ]:
## Just showing that all answer to V33 by duckduck go are -1
Community = 'Comm.Linux_Kernel'
v27 = df[(df.Community == Community) & (df.V27 != -1)].V27
v28 = df[(df.Community == Community) & (df.V28 != -1)].V28
v29 = df[(df.Community == Community) & (df.V29 != -1)].V29
v30 = df[(df.Community == Community) & (df.V30 != -1)].V30

#### Model

To study this hypothesis, we estimate the distribution of the answers
to questions V27-V30 by participants of the Linux Kernel community. As mentioned earlier, we assume that the data form a [normal
distribution](https://en.wikipedia.org/wiki/Normal_distribution). We estimate the distribution for the answers to each question separately. We
set
[uniform](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous))
priors on the parameters of the normal distributions—from
$1$ to $5$ for the mean, and $0$ to $10$ for the standard
deviation. Finally we check whether the answers are positive to any of the questions above. That is, whether the participants reply in the range neutral, agree or strongly agree.


In [ ]:
with pm.Model() as protective_linux_kernel:
  μ27 = pm.Uniform('μ27', lower=1, upper=5)
  μ28 = pm.Uniform('μ28', lower=1, upper=5)
  μ29 = pm.Uniform('μ29', lower=1, upper=5)
  μ30 = pm.Uniform('μ30', lower=1, upper=5)

  σ27 = pm.Uniform('σ27', lower=0, upper=4)
  σ28 = pm.Uniform('σ28', lower=0, upper=4)
  σ29 = pm.Uniform('σ29', lower=0, upper=4)
  σ30 = pm.Uniform('σ30', lower=0, upper=4)

  obs27 = pm.Normal('obs27',mu=μ27,sigma=σ27,observed=v27)
  obs28 = pm.Normal('obs28',mu=μ28,sigma=σ28,observed=v28)
  obs29 = pm.Normal('obs29',mu=μ29,sigma=σ29,observed=v29)
  obs30 = pm.Normal('obs30',mu=μ30,sigma=σ30,observed=v30)

#### Sampling

In [ ]:
with protective_linux_kernel:
  trace_protective_linux_kernel = pm.sample(10000,cores=2)
  ppc_protective_linux_kernel = pm.sample_posterior_predictive(trace_protective_linux_kernel, 
                                                               samples=5000, 
                                                               model=protective_linux_kernel)

#### Plotting

We show the different distributions for the data. The plots show the percentage of probability density above and below 3, i.e., the percentage of answers above (towards strong disagree) or below (towards strong agree) neutral.

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(20,8))
pm.plot_posterior(ppc_protective_linux_kernel['obs27'],ref_val=3,point_estimate='mode', ax=axs[0,0])
pm.plot_posterior(ppc_protective_linux_kernel['obs28'],ref_val=3,point_estimate='mode', ax=axs[0,1])
pm.plot_posterior(ppc_protective_linux_kernel['obs29'],ref_val=3,point_estimate='mode', ax=axs[1,0])
pm.plot_posterior(ppc_protective_linux_kernel['obs30'],ref_val=3,point_estimate='mode', ax=axs[1,1])
axs[0, 0].set_title('Answers V27')
axs[0, 1].set_title('Answers V28')
axs[1, 0].set_title('Answers V29')
axs[1, 1].set_title('Answers V30')
plt.show()

#### Queries

##### Protective

We define a function that precisely defines whether the answers to a question are _positive_ based on its probability distribution. 

Note that for each question we consider the distribution of the answer as positive if the distribution has 95% of the asnwers below 3.0. Intuitively, it means that all the answers are concentrated in strong agree, agree or neutral. This is typically known in statistics as the Region of Practical Equivalence ([ROPE](https://cran.r-project.org/web/packages/bayestestR/vignettes/region_of_practical_equivalence.html)). In other words, our results assume an error of 5%. [NOTE: check the last statement]

In [ ]:
def protective(trace):
    return (np.mean(trace['obs28'] < 3.0) > .95 or
            np.mean(trace['obs29'] < 3.0) > .95 or
            np.mean(trace['obs30'] < 3.0) > .95)

Now we ask whether the Linux Kernel community is protective.

In [ ]:
print("Is the " + Community + " community protective? " + str(protective(ppc_protective_linux_kernel)))

#### Conclusions

As the output of the cell above suggest, the Linux Kernel communitive is not protective. Therefore we can conclude that **hypothesis 1 is false**.

The HPD intervals show that the answers are heavily spread over all possibilities, i.e., all values from 1 to 5 are within the HPD interval. This observation indicates that the community does not show a central tendency in how protective they are.

### H2: The FOSSASIA community adopts an equitable style of governance for its pull request process

#### Data

Here we simply load the data we will analyze, i.e., answers to question V31 and V32 by the FOSSASIA community.

In [ ]:
# Preview of the answers of the FOSSASIA community to answers V31 and V32
Community = 'Comm.FOSSASIA'
v31 = df[(df.Community == Community) & (df.V31 != -1)].V31
v32 = df[(df.Community == Community) & (df.V32 != -1)].V32

#### Model

To study this hypothesis, we estimate the distribution of the answers to questions V31 and V32 by participants of the FOSSASIA community. As mentioned earlier, we assume that the data form a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution). We estimate the distribution for the answers to each question separately. We set [uniform](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)) priors on the parameters of the normal distributions—from $1$ to $5$ for the mean, and $0$ to $10$ for the standard deviation. Finally we check whether the answers are positive to any of the questions above. That is, whether the participants reply in the range neutral, agree or strongly agree.

In [ ]:
with pm.Model() as equitable_fossasia:
  μ31 = pm.Uniform('μ31', lower=1, upper=5)
  μ32 = pm.Uniform('μ32', lower=1, upper=5)

  σ31 = pm.Uniform('σ31', lower=0, upper=4)
  σ32 = pm.Uniform('σ32', lower=0, upper=4)

  obs31 = pm.Normal('obs31',mu=μ31,sigma=σ31,observed=v31)
  obs32 = pm.Normal('obs32',mu=μ32,sigma=σ32,observed=v32)

#### Sampling

In [ ]:
with equitable_fossasia:
  trace_equitable_fossasia = pm.sample(10000,cores=2)
  ppc_equitable_fosassia = pm.sample_posterior_predictive(trace_equitable_fossasia, 
                                                          samples=5000, 
                                                          model=equitable_fossasia)

#### Plotting

We show the different distributions for the data. The plots show the percentage of probability density above and below 3, i.e., the percentage of answers above (towards strong disagree) or below (towards strong agree) neutral.

In [ ]:

fig, (ax1,ax2) = plt.subplots(2, figsize=(20,8))
pm.plot_posterior(ppc_equitable_fosassia['obs31'],ref_val=3,point_estimate='mode',ax=ax1)
pm.plot_posterior(ppc_equitable_fosassia['obs32'],ref_val=3,point_estimate='mode',ax=ax2)
ax1.set_title('Answers V31')
ax2.set_title('Answers V32')
plt.show()

#### Queries

##### Equitable

Here we define a function that determines whether the answers of a community are _equitable_.
To do so, the function checks whether the community is _positive_ to question V31 and V32.
We precisely defined begin positive as replying with answers in the range neutral, agree or strongly agree (see [Answers](#Answers))

Note that, in the function below, for each question we consider the distribution of the answer as positive if the distribution has 95% of the asnwers below 3.0. Intuitively, it means that all the answers are concentrated in strong agree, agree or neutral. This is typically known in statistics as the Region of Practical Equivalence ([ROPE](https://cran.r-project.org/web/packages/bayestestR/vignettes/region_of_practical_equivalence.html)). In other words, our results assume an error of 5%. [NOTE: check the last statement]

In [ ]:
def equitable(trace):
    return (np.mean(trace['obs31'] < 3.0) > .95 or
            np.mean(trace['obs32'] < 3.0) > .95)

Now we ask whether the FOSSASIA community is equitable

In [ ]:
print("Is the " + Community + " community equitable? " + str(equitable(ppc_equitable_fosassia)))

#### Conclusion

As stated in the previous cell, the results indicate that the FOSSASIA
community displays an equitable behaviour. Therefore, **hypothesis H2
is true**.

By looking a the graphs in the plotting section, we can see that the
hypothesis is true due to the answers to question V32. This
distribution has mode 1.3 and the HDP interval goes from 0.2 to 2.3,
meaning that most participants answer strongly agree or
agree. Nevertheless, the answers to question V31 are closer to
neutral. The HDP interval goes from 0 to 3.3, also 6.3% of the
distribution is in the range of answers neutral/disagree. This might
suggest that there are some participants who responded  "disagree"
to this question.

### H4: The DuckDuckGo community adopts a lenient style of governance for its pull request process


#### Data

Here we simply print the data we will analyze, i.e., answers to question V33 by the Coala community.

In [ ]:
## Just showing that all answer to V33 by duckduck go are -1
df[(df.Community == 'Comm.DuckDuckGo')].V33

#### Conclusions

Since nobody from the DuckDuckGo community reply to answer V33 we cannot perform the analysis.

### H5: The Coala community adopts a linient style of governance for its pull request process


#### Data

Here we simply load the data we will analyze, i.e., answers to question V33 by the Coala community.

In [ ]:
v33_Coala = df[(df.Community == 'Comm.Coala') & (df.V33 != -1)].V33

#### Model

To study this hypothesis, we estimate the distribution of the answers
to question V33 by participants of the Coala community. As mentioned earlier, we assume that the data form a [normal
distribution](https://en.wikipedia.org/wiki/Normal_distribution). We
set
[uniform](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous))
priors on the parameters of the normal distributions for the data—from
$1$ to $5$ for the mean, and $0$ to $10$ for the standard
deviation. Finally we look into the posterior distribution to determine whether the hypothesis is validated by the data.


In [ ]:
with pm.Model() as coala_lenient:
    μ = pm.Uniform('μ', 1, 5)
    σ = pm.Uniform('σ', 0, 10)

    obs = pm.Normal('obs', mu=μ, sigma=σ,
                    observed=v33_Coala)

#### Sampling

In [ ]:
with coala_lenient:
  trace_coala_lenient = pm.sample(10000,cores=2) # Compute posterior
  ppc_coala_lenient = pm.sample_posterior_predictive(trace_coala_lenient,
                                                     samples=5000,
                                                     model=coala_lenient) # Computer predictive check

#### Plotting

We show the distribution for the data. The plot shows the percentage of probability density above and below 3, i.e., the percentage of answers above (towards strong disagree) or below (towards strong agree) neutral.

In [ ]:

fig, ax = plt.subplots()
pm.plot_posterior(ppc_coala_lenient['obs'], ref_val=3, rope=(1.5,2.5), ax=ax)
ax.set_title('Answers V33')
plt.show()

#### Conclusions

The HDP of the distribution is from 0.4 to 3.1, i.e., most answers go from strongly agree to neutral. Only 3.9% of the answers are greater than 3, i.e., only 3.9% of the answer are on the side of disagree of strongly disagree. Therefore we can conclude that **hypothesis H5 is true**.

### H7 (BONUS): The Coala Community is more lenient than the Linux Kernel Community

#### Data

Here we simply load the data we will analyze, i.e., answers to
question V33 by the Coala and Linux Kernel community.

In [ ]:
v33_Coala = df[(df.Community == 'Comm.Coala') & (df.V33 != -1)].V33
v33_Linux_Kernel = df[(df.Community == 'Comm.Linux_Kernel') & (df.V33 != -1)].V33

#### Model

To study this hypothesis, we estimate the distributions of the answers
to question V33 by the Coala and Linux Kernel communities.
As mentioned earlier, we assume that the data form a [normal
distribution](https://en.wikipedia.org/wiki/Normal_distribution). We
set
[uniform](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous))
priors on the parameters of the normal distributions for the data—from
$1$ to $5$ for the mean, and $0$ to $10$ for the standard
deviation.

In order to compare the answers, we additionally compute the different
between the mean and standard deviation of the two estimated
distributions. Moreover, we compute the effective size between the two
differences. The effective size is a standard method to compared to
compare two estimated normal distributions (see puppies Chapter 16).


In [ ]:
with pm.Model() as lenient_coala_vs_linuxkernel:
    μ_Coala = pm.Uniform('μ_Coala', 1, 5)
    σ_Coala = pm.Uniform('σ_Coala', 0, 10)

    obs_Coala = pm.Normal('obs_Coala', mu=μ_Coala, sigma=σ_Coala, observed=v33_Coala)

    μ_Linux_Kernel = pm.Uniform('μ_Linux_Kernel', 1, 5)
    σ_Linux_Kernel = pm.Uniform('σ_Linux_Kernel', 0, 10)

    obs_Linux_Kernel = pm.Normal('obs_Linux_Kernel', mu=μ_Linux_Kernel, sigma=σ_Linux_Kernel, observed=v33_Linux_Kernel)

    μ_diff = pm.Deterministic('μ_diff',μ_Coala-μ_Linux_Kernel)
    σ_diff = pm.Deterministic('σ_diff',σ_Coala-σ_Linux_Kernel)
    # See Chapter 16 of the puppies for definition of effect size
    eff_size = pm.Deterministic('eff_size', (μ_Coala-μ_Linux_Kernel)/pm.math.sqrt((pm.math.sqr(σ_Coala)+pm.math.sqr(σ_Linux_Kernel))/2))

#### Sampling

In [ ]:
with lenient_coala_vs_linuxkernel:
  trace_lenient_coala_vs_linuxkernel = pm.sample(10000,cores=2) # Compute posterior
  ppc_lenient_coala_vs_linuxkernel = pm.sample_posterior_predictive(trace_lenient_coala_vs_linuxkernel,
                                                                    samples=5000,
                                                                    model=lenient_coala_vs_linuxkernel) # Computer predictive check

#### Plotting

We show the different distributions for the data. The plots show the percentage of probability density above and below 3, i.e., the percentage of answers above (towards strong disagree) or below (towards strong agree) neutral.

Furthermore, we show the distributons on the difference of means, standard deviation and effective size. These plots provide useful information in determining whether the distributions on the answers are different.

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
pm.plot_posterior(ppc_lenient_coala_vs_linuxkernel['obs_Coala'], ref_val=3, ax=ax1)
pm.plot_posterior(ppc_lenient_coala_vs_linuxkernel['obs_Linux_Kernel'], ref_val=3, ax=ax2)
ax1.set_title('Answers V33 Coala')
ax2.set_title('Answers V33 Linux Kernel')
pm.plot_posterior(trace_lenient_coala_vs_linuxkernel, var_names=['μ_diff','σ_diff','eff_size'], ref_val=0, rope=(-0.1,0.1))
plt.show()

#### Conclusion

The differences of means $\mu\_\mathit{diff}\,$ indicates that there is
a difference of 1.2 towards more lenient answers in the Coala
community. Furthermore, the posterior distribution of the effect size
suggest a nonzero difference well outside of a (-0.1,0.1) ROPE.

Consequently we can conclude that **hypothesis H7 is true**.


## Results Summary

| Hypotehsis | Results | Observations                                |
|------------|---------|---------------------------------------------|
| H1         | False   |                                             |
| H2         | True    |                                             |
| H3         | —       | To analyze                                  |
| H4         | ???     | There are no answers to V33 from DuckDuckGo |
| H5         | True    |                                             |
| H6         | —       | To analyze                                  |
| H7(BONUS)  | True    |                                             |


## What comes below is just random experiments not part of the report

[NOTE: Ignore this cell, I simply use it as a template when I run an experiment. That said, it gives a good overview of the structure of each section.]
### HX Title of hypothesis
#### Data
#### Model
#### Sampling
#### Plotting
#### Queries
#### Conclusion

In [ ]:
## Protective Coala?
### If answers are negative to V28-V30 does it mean that the community
### is not protective?

##### NOTE: Positive is > agree (also after neutral)

Community = 'Comm.ROS'
v28 = df[(df.Community == Community) & (df.V28 != -1)].V28
v29 = df[(df.Community == Community) & (df.V29 != -1)].V29
v30 = df[(df.Community == Community) & (df.V30 != -1)].V30
v31 = df[(df.Community == Community) & (df.V31 != -1)].V31
v32 = df[(df.Community == Community) & (df.V32 != -1)].V32

with pm.Model() as model:
    μ28 = pm.Uniform('μ28', lower=1, upper=5)
    μ29 = pm.Uniform('μ29', lower=1, upper=5)
    μ30 = pm.Uniform('μ30', lower=1, upper=5)
    μ31 = pm.Uniform('μ31', lower=1, upper=5)
    μ32 = pm.Uniform('μ32', lower=1, upper=5)

    σ28 = pm.Uniform('σ28', lower=0, upper=4)
    σ29 = pm.Uniform('σ29', lower=0, upper=4)
    σ30 = pm.Uniform('σ30', lower=0, upper=4)
    σ31 = pm.Uniform('σ31', lower=0, upper=4)
    σ32 = pm.Uniform('σ32', lower=0, upper=4)

    obs28 = pm.Normal('obs28',mu=μ28,sigma=σ28,observed=v28)
    obs29 = pm.Normal('obs29',mu=μ29,sigma=σ29,observed=v29)
    obs30 = pm.Normal('obs30',mu=μ30,sigma=σ30,observed=v30)
    obs31 = pm.Normal('obs31',mu=μ31,sigma=σ31,observed=v31)
    obs32 = pm.Normal('obs32',mu=μ32,sigma=σ32,observed=v32)

In [ ]:
with model:
    trace = pm.sample(10000,cores=2)

In [ ]:
pm.traceplot(trace)

In [ ]:
## TO DISCUSS: I put as a condition > .95 instead of == 1.0 because ==
## 1.0 is hard to get. Also we should adjust for noise.

def protective(trace):
    return (np.mean(trace['obs28'] < 3.0) > .95 or
            np.mean(trace['obs29'] < 3.0) > .95 or
            np.mean(trace['obs30'] < 3.0) > .95)

def equitable(trace):
    return (np.mean(trace['obs31'] < 3.0) > .95 or
            np.mean(trace['obs32'] < 3.0) > .95)

In [ ]:
ppc = pm.sample_posterior_predictive(trace, samples=5000, model=model)

In [ ]:
pm.plot_posterior(ppc['obs29'],ref_val=3)

In [ ]:
pm.plot_posterior(trace['μ28'],ref_val=3)

In [ ]:
print("The community " + Community + " is" + ("" if protective(ppc) else " not") + " protective")
print("The community " + Community + " is" + ("" if equitable(ppc) else " not") + " equitable")

In [ ]:
print(np.mean(ppc['obs28']<3))
print(np.mean(ppc['obs29']<3))
print(np.mean(ppc['obs30']<3))
print(np.mean(ppc['obs31']<3))
print(np.mean(ppc['obs32']<3))